In [6]:
import os
import sys
sys.path.insert(0,'..')

from bidst.transformers import SkullStrippingTransformer
from bidst.transformers import TissueSegmentation
from bidst.transformers import StatAggregator
from bidst.pipeline import BIDSPipeline


In [7]:
!tree -a ../bidst/tests/data/ds114/

../bidst/tests/data/ds114/
├── derivatives
│   └── test_pipeline
│       └── steps
│           ├── skullstripping
│           │   ├── sub-02
│           │   │   ├── ses-retest
│           │   │   │   └── anat
│           │   │   │       ├── sub-02_ses-retest_T1w.nii.gz
│           │   │   │       └── sub-02_ses-retest_T1w_brain.nii.gz
│           │   │   └── ses-test
│           │   │       └── anat
│           │   │           └── sub-02_ses-test_T1w_brain.nii.gz
│           │   └── sub-03
│           │       ├── ses-retest
│           │       │   └── anat
│           │       │       └── sub-03_ses-retest_T1w_brain.nii.gz
│           │       └── ses-test
│           │           └── anat
│           │               └── sub-03_ses-test_T1w_brain.nii.gz
│           └── tissuesegmentation
│               ├── sub-02
│               │   ├── ses-retest
│               │   │   └── anat
│               │   │       ├── sub-02_ses-retest_T1w_brain_class-CSF_probtissue.nii.gz
│               │   │

In [8]:
import nibabel as nib
from functools import reduce
from operator import mul

def get_tissue_stats(subject_folder):
    
    ext_dict = {}
    ext_dict['WM_volume'] = 'WM_probtissue'
    ext_dict['GM_volume'] = 'GM_probtissue'
    ext_dict['CSF_volume'] = 'CSF_probtissue'
    
    files_list = os.listdir(subject_folder)
    
    results = {}
    for key in ext_dict.keys():
        cur_file = [item for item in files_list if ext_dict[key] in item]
        if len(cur_file) > 1:
            raise ValueError('Several candiadates in the folder')
        else:
            cur_file = cur_file[0]
        cur_img = nib.load(subject_folder + '/' + cur_file)
        voxel_sum = cur_img.get_data()
        voxel_sum = (voxel_sum >= 0.5).sum()
        voxel_vol = reduce(mul, cur_img.header.get_zooms())
        all_vol = voxel_sum * voxel_vol
        results[key] = all_vol
    
    return results

In [10]:
project_path = os.path.abspath('..') + '/bidst/tests/data/ds114'
pipeline_name = 'test_pipeline'
skullstripping = SkullStrippingTransformer(pipeline_name=pipeline_name,
                                           project_path=project_path,
                                           gather_steps=('source', dict(extensions='T1w.nii.gz')))
tissuesegmentation = TissueSegmentation(pipeline_name=pipeline_name,
                                        project_path=project_path,
                                        gather_steps=('skullstripping', dict(extensions='T1w_brain.nii.gz')))

aggregator = StatAggregator(function=get_tissue_stats,
                            project_path=project_path,
                            pipeline_name=pipeline_name,
                            gather_steps=('tissuesegmentation', dict(extensions='WM_probtissue.nii.gz')))

IDS = ['03']


pipe = BIDSPipeline(steps=[('skullstripping', skullstripping),
                           ('tissuesegmentation', tissuesegmentation),
                           ('aggregator', aggregator)],
             pipeline_name=pipeline_name, project_path=project_path)

In [ ]:
pipe.fit_transform(IDS)

In [5]:
!tree -a ../bidst/tests/data/ds114/

../bidst/tests/data/ds114/
├── derivatives
│   └── test_pipeline
│       └── steps
│           ├── skullstripping
│           │   ├── sub-02
│           │   │   ├── ses-retest
│           │   │   │   └── anat
│           │   │   │       ├── sub-02_ses-retest_T1w.nii.gz
│           │   │   │       └── sub-02_ses-retest_T1w_brain.nii.gz
│           │   │   └── ses-test
│           │   │       └── anat
│           │   │           └── sub-02_ses-test_T1w_brain.nii.gz
│           │   └── sub-03
│           │       ├── ses-retest
│           │       │   └── anat
│           │       │       └── sub-03_ses-retest_T1w_brain.nii.gz
│           │       └── ses-test
│           │           └── anat
│           │               └── sub-03_ses-test_T1w_brain.nii.gz
│           └── tissuesegmentation
│               ├── sub-02
│               │   ├── ses-retest
│               │   │   └── anat
│               │   │       ├── sub-02_ses-retest_T1w_brain_class-CSF_probtissue.nii.gz
│               │   │